## Data Preparation & Exploration for Climate (ulmo)

I want to merge the data from GSOD to my ILI data by getting a summary of the data by position
-  exploring daymet : daily temperature, precipitation for any locationsion in the US [reference](https://github.com/ulmo-dev/ulmo/blob/master/examples/Using%20Daymet%20weather%20data%20from%20ORNL%20webservice.ipynb)

In [1]:
# import packages and modules
from ulmo.nasa import daymet
from delphi_epidata import Epidata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.utils import shuffle

pd.set_option('display.max_columns', 100)

/anaconda/lib/python3.6/site-packages/ulmo/twc/kbdi/core.py:20: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  CSV_SWITCHOVER = pandas.tslib.Timestamp('2016-10-01')


In [7]:
#test
ili_test = pd.read_csv("../data/raw.csv")
ili_test.head()

,statename,activity_level,activity_level_label,weekend,season,weeknumber,Latitude,Longitude
0,Alabama,1,Minimal,Oct-01-2016,2015-16,39,32.806671,-86.79113
1,Alabama,1,Minimal,Sep-24-2016,2015-16,38,32.806671,-86.79113
2,Alabama,1,Minimal,Sep-17-2016,2015-16,37,32.806671,-86.79113
3,Alabama,1,Minimal,Sep-10-2016,2015-16,36,32.806671,-86.79113
4,Alabama,1,Minimal,Sep-03-2016,2015-16,35,32.806671,-86.79113


In [24]:
coordinate = ili_test[['statename','Latitude','Longitude']]
coordinate = coordinate.drop_duplicates()
coordinate = coordinate.reset_index(drop=True)
coordinate.head()
#print("the data contains {} rows".format(len(coordinate)))

,statename,Latitude,Longitude
0,Alabama,32.806671,-86.791130
1,Alaska,61.370716,-152.404419
2,Arizona,33.729759,-111.431221
3,Arkansas,34.969704,-92.373123
4,California,36.116203,-119.681564


In [25]:
coordinate.Longitude[10]

-83.643073999999999

In [27]:
climate = []
for i in range(2,4):
    df = daymet.get_daymet_singlepixel(longitude=coordinate.Longitude[i], latitude=coordinate.Latitude[i], 
                                   years=[2010,2015])
    df['statename'] = coordinate.statename[i]
    df['Latitude'] = coordinate.Latitude[i]
    df['Longitude'] = coordinate.Longitude[i]
    climate.append(df)
    
climate = pd.concat(climate)
climate

making request for latitude, longitude: 33.729759, -111.431221
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=33.729759&lon=-111.431221&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=33.729759&lon=-111.431221&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015
making request for latitude, longitude: 34.969704, -92.37312299999999
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=34.969704&lon=-92.37312299999999&measuredParams=tmax,tmin,prcp&year=2010,2015&lat=34.969704&lon=-92.373123&vars=tmax%2Ctmin%2Cprcp&years=2010%2C2015


,year,yday,prcp,tmax,tmin,statename,Latitude,Longitude
2010-01-01,2010,1,0.0,15.0,2.0,Arizona,33.729759,-111.431221
2010-01-02,2010,2,0.0,17.0,2.5,Arizona,33.729759,-111.431221
2010-01-03,2010,3,0.0,17.0,2.5,Arizona,33.729759,-111.431221
2010-01-04,2010,4,0.0,16.5,3.5,Arizona,33.729759,-111.431221
2010-01-05,2010,5,0.0,17.0,3.0,Arizona,33.729759,-111.431221


In [29]:
climate

,year,yday,prcp,tmax,tmin,statename,Latitude,Longitude
2010-01-01,2010,1,0.0,15.0,2.0,Arizona,33.729759,-111.431221
2010-01-02,2010,2,0.0,17.0,2.5,Arizona,33.729759,-111.431221
2010-01-03,2010,3,0.0,17.0,2.5,Arizona,33.729759,-111.431221
2010-01-04,2010,4,0.0,16.5,3.5,Arizona,33.729759,-111.431221
2010-01-05,2010,5,0.0,17.0,3.0,Arizona,33.729759,-111.431221
2010-01-06,2010,6,0.0,17.0,3.5,Arizona,33.729759,-111.431221
2010-01-07,2010,7,0.0,16.0,4.0,Arizona,33.729759,-111.431221
2010-01-08,2010,8,0.0,14.5,4.0,Arizona,33.729759,-111.431221
2010-01-09,2010,9,0.0,17.0,1.5,Arizona,33.729759,-111.431221
2010-01-10,2010,10,0.0,15.5,2.5,Arizona,33.729759,-111.431221


In [13]:


res = Epidata.fluview(['nat'], [201440, Epidata.range(201501, 201510)])
print(res['result'], res['message'], len(res['epidata']))

1 success 11


In [128]:
#test
ornl_lat, ornl_long = 35.9313167, -84.3104124
df = daymet.get_daymet_singlepixel(longitude=ornl_long, latitude=ornl_lat, 
                                   years=[2012,2013])


making request for latitude, longitude: 35.9313167, -84.3104124
processing data from request: https://daymet.ornl.gov/data/send/saveData?lat=35.9313167&lon=-84.3104124&measuredParams=tmax,tmin,prcp&year=2012,2013&lat=35.9313167&lon=-84.3104124&vars=tmax%2Ctmin%2Cprcp&years=2012%2C2013


In [129]:
df.index.year

Int64Index([2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012, 2012,
            ...
            2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013, 2013],
           dtype='int64', length=730)

In [130]:
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
#df.drop('index', axis=0, inplace=True)
df.head()

,year,yday,prcp,tmax,tmin,Latitude,Longitude,month,day
2012-01-01,2012,1,0.0,15.5,1.0,35.931317,-84.310412,1,1
2012-01-02,2012,2,3.0,10.0,-3.0,35.931317,-84.310412,1,2
2012-01-03,2012,3,0.0,2.0,-7.0,35.931317,-84.310412,1,3
2012-01-04,2012,4,0.0,4.5,-8.0,35.931317,-84.310412,1,4
2012-01-05,2012,5,0.0,10.0,-4.0,35.931317,-84.310412,1,5


[Pandas dataframe groupeby datetime month](https://stackoverflow.com/questions/24082784/pandas-dataframe-groupby-datetime-month)


In [131]:
df.shape

(730, 9)

In [132]:
# Group the data by month, and take the mean for each group (i.e. each month)
df[['prcp', 'tmax', 'tmin']].resample('M').mean().add_prefix('mean_')

,mean_prcp,mean_tmax,mean_tmin
2012-01-31,5.548387,11.193548,0.000000
2012-02-29,3.413793,13.189655,1.379310
2012-03-31,4.193548,22.532258,8.580645
2012-04-30,3.866667,23.066667,9.066667
2012-05-31,3.290323,28.000000,15.564516
2012-06-30,1.533333,30.833333,16.883333
2012-07-31,7.193548,32.967742,21.112903
2012-08-31,3.709677,30.338710,18.306452
2012-09-30,7.266667,27.166667,15.033333
2012-10-31,1.741935,19.774194,8.645161


In [133]:
df_month = df[['month','year', 'yday', 'prcp', 'tmax', 'tmin']].groupby(['year', 'month']).mean()
df_month.head()

yday      prcp       tmax       tmin
year month                                       
2012 1       16.0  5.548387  11.193548   0.000000
     2       46.0  3.413793  13.189655   1.379310
     3       76.0  4.193548  22.532258   8.580645
     4      106.5  3.866667  23.066667   9.066667
     5      137.0  3.290323  28.000000  15.564516

In [134]:
df_month.shape

(24, 4)

We have :
-  categorical variable : year, yday(integer)
-  other variable : floats or interger
-  index : datetime

In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2012-01-01 to 2013-12-31
Data columns (total 9 columns):
year         730 non-null int64
yday         730 non-null int64
prcp         730 non-null float64
tmax         730 non-null float64
tmin         730 non-null float64
Latitude     730 non-null float64
Longitude    730 non-null float64
month        730 non-null int64
day          730 non-null int64
dtypes: float64(5), int64(4)
memory usage: 57.0 KB


In [75]:
#load ILI data from csv
ili = pd.read_csv("../data/raw.csv")
ili.head()

,statename,activity_level,activity_level_label,weekend,season,weeknumber,Latitude,Longitude
0,Alabama,1,Minimal,Oct-01-2016,2015-16,39,32.806671,-86.79113
1,Alabama,1,Minimal,Sep-24-2016,2015-16,38,32.806671,-86.79113
2,Alabama,1,Minimal,Sep-17-2016,2015-16,37,32.806671,-86.79113
3,Alabama,1,Minimal,Sep-10-2016,2015-16,36,32.806671,-86.79113
4,Alabama,1,Minimal,Sep-03-2016,2015-16,35,32.806671,-86.79113


In [76]:
# Clean the data 
# tramsform to datetime
ili['weekend'] = pd.to_datetime(ili['weekend'], format='%b-%d-%Y')
ili['season'] = pd.to_datetime(ili['season'], format='%Y-%y') 

# add year month and day in the data
ili['year'] = ili.weekend.dt.year
ili['month'] = ili.weekend.dt.month
ili['day'] = ili.weekend.dt.day

#remove data that we don't need
ili.drop(['weekend','season','weeknumber'], axis=1, inplace=True)

In [70]:
ili.head()

,statename,activity_level,activity_level_label,Latitude,Longitude,year,month,day
0,Alabama,1,Minimal,32.806671,-86.79113,2016,10,1
1,Alabama,1,Minimal,32.806671,-86.79113,2016,9,24
2,Alabama,1,Minimal,32.806671,-86.79113,2016,9,17
3,Alabama,1,Minimal,32.806671,-86.79113,2016,9,10
4,Alabama,1,Minimal,32.806671,-86.79113,2016,9,3


In [77]:
ili.shape

(15933, 8)

In [78]:
ili.drop_duplicates()
ili.shape

(15933, 8)

In [79]:
ili.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15933 entries, 0 to 15932
Data columns (total 8 columns):
statename               15933 non-null object
activity_level          15933 non-null int64
activity_level_label    15933 non-null object
Latitude                15933 non-null float64
Longitude               15933 non-null float64
year                    15933 non-null int64
month                   15933 non-null int64
day                     15933 non-null int64
dtypes: float64(2), int64(4), object(2)
memory usage: 995.9+ KB


## Metadata

store meta-information about the variables in a DataFrame


-  **role**: response, explanatory (variable that we want to predict)
-  **level**: nominal, interval, ordinal, binary
-  **keep**: True or False
-  **dtype**: int, float, str

In [56]:
meta_ili = []
for f in ili.columns:
    # Defining the role
    if f == 'activity_level' or f == 'activity_level_label':
        role = 'response'
    else:
        role = 'explanatory'
         
    # Defining the level
    if 'statename' == f :
        level = 'nominal'
    elif 'activity_level_label' == f:
        level = 'ordinal'
    elif 'weekend' == f or 'season' == f or 'weeknumber' == f:
        level = 'interval'
    elif ili[f].dtype == float:
        level = 'ordinal'
    elif ili[f].dtype == int:
        level = 'ordinal'
        
    # Initialize keep to True for all variables except for id
    keep = True
    if f == 'activity_level_label':
        keep = False
    
    # Defining the data type 
    dtype = ili[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'role': role,
        'level': level,
        'keep': keep,
        'dtype': dtype
    }
    meta_ili.append(f_dict)
    
meta = pd.DataFrame(meta_ili, columns=['varname', 'role', 'level', 'keep', 'dtype'])
meta.set_index('varname', inplace=True)

In [57]:
meta

,role,level,keep,dtype
varname,,,,
statename,explanatory,nominal,True,object
activity_level,response,ordinal,True,int64
activity_level_label,response,ordinal,False,object
weekend,explanatory,interval,True,datetime64[ns]
season,explanatory,interval,True,datetime64[ns]
weeknumber,explanatory,interval,True,int64
Latitude,explanatory,ordinal,True,float64
Longitude,explanatory,ordinal,True,float64


In [58]:
# Below the number of variables per role and level are displayed.

pd.DataFrame({'count' : meta.groupby(['role', 'level'])['role'].size()}).reset_index()

,role,level,count
0,explanatory,interval,3
1,explanatory,nominal,1
2,explanatory,ordinal,2
3,response,ordinal,2


### interval variables

In [59]:
v = meta[(meta.level == 'ordinal') & (meta.keep)].index
ili[v].describe()

,activity_level,Latitude,Longitude
count,15933.000000,15933.000000,15933.000000
mean,1.957510,39.465562,-93.366505
std,2.183754,6.015489,19.102700
min,0.000000,21.094318,-157.498337
25%,1.000000,35.630066,-105.311104
50%,1.000000,39.849426,-89.616508
75%,2.000000,43.326618,-78.169968
max,10.000000,61.370716,-69.381927


In [60]:
v = meta[(meta.level == 'nominal') & (meta.keep)].index
ili[v].describe()

,statename
count,15933
unique,51
top,Alaska
freq,313


### Checking the cardinality of the categorical variables

In [62]:
v = meta[(meta.level == 'interval') & (meta.keep)].index

for f in v:
    dist_values = ili[f].value_counts().shape[0]
    print('Variable {} has {} distinct values'.format(f, dist_values))

Variable weekend has 313 distinct values
Variable season has 6 distinct values
Variable weeknumber has 53 distinct values
